In [1]:
%cd ../

/Users/eloidieme/dev/python-projects/DD2424-project


/Users/eloidieme/dev/python-projects/DD2424-project/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
def split_text(text, train_frac=0.8, val_frac=0.1):
    train_end = int(len(text) * train_frac)
    val_end = train_end + int(len(text) * val_frac)

    train_data = text[:train_end]
    val_data = text[train_end:val_end]
    test_data = text[val_end:]

    return train_data, val_data, test_data

In [3]:
from nlpProject.eval import make_val, eval_lstm1
from nlpProject.make_data import DataMaker
from nlpProject.utils import compute_loss

with open('./data/shakespeare_val.txt', 'r') as f:
    text = f.read()

data_maker = DataMaker()
X_val, Y_val = make_val(data_maker, text, 400)
X_val.shape, Y_val.shape

(torch.Size([66, 400, 278]), torch.Size([66, 400, 278]))

In [9]:
from nlpProject.one_layer_lstm import LSTM1
import torch
from pathlib import Path

state_dict = torch.load(Path(f'./models/LSTM/lstm1_256_400_100_32_0.01.pt'))
input_size, hidden_size = state_dict["Wf"].size()[1], state_dict["Wf"].size()[0]
model = LSTM1(input_size, hidden_size, input_size)
model.load_state_dict(state_dict)
model.eval()

P, Y, s = eval_lstm1(model, data_maker, X_val, Y_val, length=len(text))

111439
111440


In [18]:
with open("./reports/logs/predicted_val.txt", 'w+') as f:
    f.write(text[0] + s)

In [14]:
text

"you are,\nThat is, a woman; if you be more, you're none;\nIf you be one, as you are well express'd\nBy all external warrants, show it now,\nBy putting on the destined livery.\n\nISABELLA:\nI have no tongue but one: gentle my lord,\nLet me entreat you speak the former language.\n\nANGELO:\nPlainly conceive, I love you.\n\nISABELLA:\nMy brother did love Juliet,\nAnd you tell me that he shall die for it.\n\nANGELO:\nHe shall not, Isabel, if you give me love.\n\nISABELLA:\nI know your virtue hath a licence in't,\nWhich seems a little fouler than it is,\nTo pluck on others.\n\nANGELO:\nBelieve me, on mine honour,\nMy words express my purpose.\n\nISABELLA:\nHa! little honour to be much believed,\nAnd most pernicious purpose! Seeming, seeming!\nI will proclaim thee, Angelo; look for't:\nSign me a present pardon for my brother,\nOr with an outstretch'd throat I'll tell the world aloud\nWhat man thou art.\n\nANGELO:\nWho will believe thee, Isabel?\nMy unsoil'd name, the austereness of my life,

In [10]:
import torch
compute_loss(Y.view_as(P), P)

tensor(132015.5470, dtype=torch.float64)

In [8]:
Y = Y.t()
n_chars, n_features = Y.shape
n_samples = n_chars // Y_val.shape[1]
Y_trunc = Y[:n_samples * Y_val.shape[1]]
Y_resh = Y_trunc.view(n_samples, Y_val.shape[1], n_features).permute(2, 1, 0)

In [9]:
Y_resh.shape

torch.Size([66, 100, 1115])

In [ ]:
compute_loss()

In [4]:
eval_lstm1("models/LSTM/lstm1_256_400_100_32_0.01.pt", "data/shakespeare.txt", "data/shakespeare_val.txt", 400, 0.7)

111139
111140
------------------------------------------------
Prediction loss: 101587.7366
Precision: 90.21% - Recall: 96.93% - F-measure: 93.45% - BLEU: 11.48%
------------------------------------------------


(101587.73658986081,
 90.21058882645836,
 96.92745656228526,
 93.44847982964627,
 11.476548194885254)